<a href="https://colab.research.google.com/github/mraskj/css_fall2023/blob/main/class10_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class 10: Audio Measurement - Solution

In this exercise, we explore how we can use a Python-binding of Praat, a speech analysis software designed to study phonetics (the sound of speeches) using computers, to study acoustic features of political speeches.



## 0 Setup

We start by:

1. Cloning the course GitHub repo
2. Install necessary packages
3. Importing necessary modules
4. Reading data




### 0.1 Cloning GitHub Repository

In [ ]:
# Clone GitHub directory into
!git clone https://github.com/mraskj/css_fall2023.git

### 0.2 Install Packages

In [ ]:
# Install packages
!pip install -r /content/css_fall2023/requirements/requirements_topic4-class9.txt

### 0.3 Importing Modules

In [ ]:
# For file and directory management
import os

# For regular expressions
import re

# For data handling
import numpy as np
import pandas as pd

# For plotting
import seaborn as sns
import matplotlib.pyplot as plt
sns.set() # Use seaborn's default style
# Feature extraction
import librosa

# For Praat
import parselmouth

### 0.4 Reading Data

In this exercise, we will work with 20 audio files each corresponding to a speech in the Danish parliament in the second assembly of the 2007 parliamentary term (*20072*). The files come from 10 different speakers with two speeches each. FIve of the speakers are men and Five are women.

You find the files in *data/audio/class10/* folder in the cloned GitHub repo.

In [ ]:
# Define path to directory containing audio files
base_dir = os.path.join(os.getcwd(), 'css_fall2023/data/audio/class10')

male_files = os.listdir(os.path.join(base_dir, 'male'))
female_files = os.listdir(os.path.join(base_dir, 'female'))

If we print `male_files` and `female_files`, we'll see that they are unsorted. To keep things in order, we sort the files by the first digits in the filenames. We define a custom function for this purpose and provide the function as parameter to the `key` argument in the `sorted()` function.

In [ ]:
# Define a custom sorting key function
def digit_sort_key(v):
    digits = re.search('\d+', v)
    return int(digits.group())

# Sort the lists
male_files = sorted(male_files, key=digit_sort_key)
female_files = sorted(female_files, key=digit_sort_key)

Each audio file corresponds to speech given in a parliamentary debate in the Danish Parliament. In the GitHub repo, you find a dataset with metadata for each file: `class10_audio_df.csv`. Read in the data to an object called `speech_df`. It contains metadata for the audio files such as the name of the file, the speaker, and a bunch of other variables. Before we proceed, we divide `speech_df`, into two: `female_speech_df` and `male_speech_df`.

In [ ]:
# Read in dataframes
speech_df = pd.read_csv('/content/css_fall2023/data/class10_audio_df.csv')

# Divide dataset into two - note that we can also filter the data based on the `gender` column
female_speech_df = speech_df.iloc[:10]
male_speech_df = speech_df.iloc[10:]

## Exercise 1: Visualization of Pitch

While `parselmouth-praat` enables us to measure a wide range of acoustic features, we focus on the fundamental frequency ($F0$) here. Feel free to play around with others as well.

Recall from the lecture that $F0$ refers to the number of times per second our vocal cords vibrate when making voiced sounds. As the name suggests, voiced sounds refer to sounds that make use of our vocal cords. All vowel sounds are voiced, but not all consonants are. Some consonant sounds, such as /p/, /f/ and /s/ are unvoiced. This means the vocal cords do not vibrate when producing these sounds.

For our purpose, we are talking about $F0$ at a higher level than vowels and consonants, typically at the speech level, but other semantic units such as sentences are also likely or fixed temporal segments (e.g. a five second window). Regardless of the level of measurement, the fundamental frequency is called the *fundamental* since it's the dominant frequency in the given time period that we consider.

As we saw on the slides, the $F0$ is a strong and robust indicator of emotional arousal. The idea is simple. When a speaker becomes emotional aroused, or if a speaker wants to *project* arousal, $F0$ increases. The link is not necessarily causal, but what matters to us is that arousal is reflected in $F0$. We will explore this idea in this exercise by estimating the pitch of speeches.

The fundamental frequency of a sound is closely correlated with pitch. Although some may think that $F0$ is just a fancy word for pitch, others argue that pitch is a different concept. In fact, pitch is more to do with a person's perception of a $F0$. In other words, fundamental frequency refers to the actual physical properties of a sound wave signal, whereas pitch refers to how our ears and brains perceive the signal regarding the rate of frequency.

In this exercise, we will investigate whether we can interpret a speaker's level of emotional arousal by computing $F0$ for the 20 audio files and projecting the estimates onto a spectrogram. We did something similar in the tutorial. Your task is to:


>  **Plot each speech as a spectrogram, compute $F0$ and project it onto the spectrogram**.

1. Make two plots, one of female speakers and one for male speakers. Arrange the plots in a $5\times 2$ grid with each row being a single speaker.
2. Describe your modeling choices.

    a) How do you handle pitch estimates that are zero?

    b) Which settings do you use to compute the pitch?

    c) How do you control the time range? (see step 3)

    d) Do you restrict the y-axis of the spectrogram? Can be done by explictly passing a value to the `maximum_frequency` argument in the `parselmouth.Sound.to_spectrogram()` method.
3. Conduct step 1 for different time ranges and describe the differences.

    - Range 1: Full speech
    - Range 2: 15 seconds of speech
    - Range 3: 5 seconds of speech
    - Range 4: 2 seconds of speech



4. Based on the results, can you visually identify the speech where the speaker is most aroused? Why, why not? Does the time range matters?

## Exercise 2: Standardized vs Unstandardized Pitch

In *Exercise 1*, we visualized pitch contours across the $20$ speeches comparing two speeches for each speaker at a time. This pairwise-comparison indirectly parses out any speaker heterogeneity since we compare speeches within the same speaker.

To see why this is important, we will now compute the speech-level average of $F0$ for each speech to investigate the impact of speaker heterogeneity when interpreting results.  



> **Task 1: Compute the speech-level average $F0$ for both male and female speakers. Add metadata from the `speech_df` and combine into a dataframe. Take the columns 'sppid', 'gender' and 'spitch'. The 'sppid' column is the speaker identifier containing the name of the speaker and the party. The 'gender' column is the gender of the speaker. And the `spitch' column contains standardized pitch estimates for that specific speech computed over speeches from 2000-2021. Hence, hundreds and even thousands of speeches contribute to the score. Recall that standardization scales data to mean zero and unit variance. Sort the computed F0 means from highest to lowest and describe the results. Pay attention to the difference between your F0 estimates and the standardized estimates.**

> **Task 2: Plot the pitch estimates for the first speech for each speaker of your $20$ files. Make a total of three plots. The first plot should contain pitch distributions for both male and female speakers in one. The second plot should contain pitch distributions for female speakers only. And the third plot should contain pitch distributions for male speakers only. Arrange the plot in a grid where you have the second and third plot vertically stacked. Describe the results.**




